## Homework 3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



Letizia Russo, Daniel Losada Molina and Michele Luca Puzzo

In [ ]:
from google.colab import drive

In [ ]:
from google.colab import files
import time
import os
import datetime

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/ADM-HW3/cartelle'
urls = '/content/drive/MyDrive/ADM-HW3/urls.txt'


## Data Collection

Since no dataset was provided for this homework, we had to create our own dataset by scraping the given website, which was done in the following steps.

## 1.1 Get the list of animes

Firstly, we had to get the list of all the anime we wanted to add to our dataset. This included all the anime found in the first 400 pages. Therefore, we created a function that scrapes a certain number of pages from an initial url, to extract the url of each of them. The initial url and the following number of pages are given as input, while the resulting urls are stored in a txt file called "urls.txt" .

In [ ]:
from google.colab import files
files.upload()

Saving funzioni.py to funzioni.py


{'funzioni.py': b'#!/usr/bin/env python3\r\n# -*- coding: utf-8 -*-\r\n"""\r\nCreated on Mon Nov  8 22:26:09 2021\r\n\r\n@author: michele\r\n"""\r\nimport shutil\r\nfrom bs4 import BeautifulSoup\r\nimport datetime\r\nfrom pathlib import Path\r\nimport requests\r\nfrom tqdm import tqdm\r\nimport time\r\nimport nltk\r\nfrom nltk.corpus import stopwords\r\nfrom nltk.stem.snowball import PorterStemmer\r\nfrom nltk.tokenize import RegexpTokenizer\r\nmonths = [\'Jan\', \'Feb\', \'Mar\', \'Apr\', \'May\', \'Jun\',\'Jul\', \'Aug\', \'Sep\', \'Oct\', \'Nov\', \'Dec\']\r\n\r\ndef get_urls(init_url, number_pages):\r\n    anime = []\r\n    for page in tqdm(range(0,number_pages)):\r\n        url = init_url +str(page *50)\r\n        response = requests.get(url)\r\n        soup = BeautifulSoup(response.text, "html.parser")\r\n        for tag in soup.find_all("tr"):#cerco le righe della tabella\r\n            links = tag.find_all("a")\r\n            for link in links:\r\n                if type(link.g

In [ ]:
import funzioni
funzioni.get_urls("https://myanimelist.net/topanime.php?limit ", 400)

100%|██████████| 50/50 [00:00<00:00, 198031.35it/s]



## 1.2 Crawl books

In [ ]:
funzioni.crawl()

# 1.3 Parsing

In [ ]:
a = open(urls, "r")
enum = enumerate(a, start = 1)
d=dict((i,j) for i,j in enum)

In [ ]:
k = 9950
for i in range(200,401):
    for j in range(1,51):
      k = k + 1
      file_size = os.path.getsize(path +  "/cartella{}/page_{}.html".format(i, k))
      if file_size < 50000:
        os.remove(path +  "/cartella{}/page_{}.html".format(i, k))
        page = requests.get(d[k])
        f = open("page_{}.html".format(k), "w")
        f.write(soup.prettify())
        shutil.move("page_{}.html".format(k), path + "/cartella{}".format(i))
        f.close()



In [ ]:
path = '/content/drive/MyDrive/ADM-HW3/1-199'

In [ ]:
k = 0
for i in range(1,200):
    for j in range(1,51):
      k = k + 1
      file_size = os.path.getsize(path +  "/cartella{}/page_{}.html".format(i, k))
      if file_size < 50000:
        os.remove(path +  "/cartella{}/page_{}.html".format(i, k))
        page = requests.get(d[k])
        f = open("page_{}.html".format(k), "w")
        f.write(soup.prettify())
        shutil.move("page_{}.html".format(k), path + "/cartella{}".format(i))
        f.close()

In [ ]:
path_1 = '/content/drive/MyDrive/ADM-HW3/cartelle'

In [ ]:
import csv
import requests
import lxml
import shutil

import urllib.request
import urllib.error as uer
import time
import nltk
import datetime
import csv
import os
from bs4 import BeautifulSoup

In [ ]:
def extractData(pagePath):
    if os.path.isfile(pagePath):
        with open(pagePath, encoding="utf-8") as fp:
            soup = BeautifulSoup(fp, "html.parser")
            animeTitle = " ".join((soup.html.head.title.text.replace("- MyAnimeList.net", '')).split())

            #print("animeTitle: ", animeTitle)
            animeInfo = (soup.find_all("div",  attrs={ "class" : "spaceit_pad"}))
            #print("Animeinfo: ", animeInfo)

            aux = findField(animeInfo, "Type:")
            animeType = ""
            if aux.a != None:
                animeType = " ".join((aux.a.text).split())
            else:
                animeType = " ".join((findField(animeInfo, "Type:").findAll(text=True, recursive=False))).split()[0]
            #print("animeType :", animeType)

            numEpisodes = list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Episodes:").findAll(text=True, recursive=False)) )))[0]
            animeNumEpisode = ""
            if numEpisodes.isdigit():
                animeNumEpisode = int(numEpisodes)
            #print("animeNumEpisode: ", animeNumEpisode)

            aired = list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Aired:").findAll(text=True, recursive=False)) )))[0].split('to')
            if len(aired[0].split()) == 3:
                releaseDate = datetime.datetime(int(aired[0].split()[2]), months.index(aired[0].split()[0])+1 , int(aired[0].split()[1][:1]))
            else:
                releaseDate = ""
            if len(aired) == 2 and len(aired[1].split()) == 3:
                endDate = datetime.datetime(int(aired[1].split()[2]), months.index(aired[1].split()[0])+1 , int(aired[1].split()[1][:1]))
            else:
                endDate = ""
            #print("releaseDate: ", releaseDate)
            #print("endDate: ", endDate)

            animeNumMembers = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Members:").findAll(text=True, recursive=False)) )))[0].replace(",", ""))
            #print("animeNumMembers: ", animeNumMembers)

            try:
              animeScore = float(" ".join((soup.find_all("span", itemprop = "ratingValue")[0].text).split()))
            #print("animeScore: ", animeScore)
            except:
                animeScore = " "

            
    
            try:
              animeUsers = int(" ".join((soup.find_all("span", itemprop = "ratingCount")[0].text).split()))
            #print("animeUsers: ", animeUsers)
            except:
              animeUsers = " "
            try:
                animeRank = int(list(filter(lambda x: x[0] == '#',findField(animeInfo, "Ranked:").text.split()))[0].replace('#',''))
            #print("animeRank: ", animeRank)
            except:
                animeRank = " "
            
            animePopularity = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Popularity:").findAll(text=True, recursive=False)) )))[0].replace("#", ""))
            #print("animePopularity: ", animePopularity)

            animeDescription = soup.find_all("p", itemprop = "description")[0].text.strip().replace('\n', '').replace('  ', '')
            #print("animeDescription: ", animeDescription)

            animeRelatedAHref = []
            if soup.find(name="table",attrs={"class":"anime_detail_related_anime"}) != None:
                animeRelatedAHref = soup.find(name="table",attrs={"class":"anime_detail_related_anime"}).findChildren('a', href=True)
            animeRelated = []
            for x in animeRelatedAHref:
                aux = " ".join((x.text).split())
                if aux not in animeRelated:
                    animeRelated.append(aux)
            #print("animeRelated: ", animeRelated)

            animeCharactersRaw = soup.find_all("h3", attrs={"class" : "h3_characters_voice_actors"})
            animeCharacters = []
            for x in animeCharactersRaw:
                animeCharacters.append(" ".join((x.text).split()))
            #print("animeCharacters: ", animeCharacters)

            animeVoicesRaw = soup.find_all("td", attrs={"class" : "va-t ar pl4 pr4"})
            animeVoices = []
            for x in animeVoicesRaw:
                animeVoices.append(" ".join((x.contents[1].text).split()))
            #print("animeVoices: ", animeVoices)

            animeStaff = ""
            if(len(soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})) == 2):
                aux = soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})[1]
                animeStaffRaw = []
                for x in aux.findChildren('a'):
                    if (" ".join((x.text).split()) != ''):
                        animeStaffRaw.append(" ".join((x.text).split()))
                animeStaffTaskRaw = []
                for x in aux.findChildren('small'):
                    animeStaffTaskRaw.append(" ".join((x.text).split()))
                #print("animeStaffRaw: ", animeStaffRaw)
                #print("animeStaffTaskRaw: ", animeStaffTaskRaw)
                animeStaff = [list(a) for a in zip(animeStaffRaw, animeStaffTaskRaw)]


            #print("animeStaff: ", animeStaff)

            return [animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff]
    return []
def findField(array, word):
    for x in array:
        #print("#####################################")
        #print(" ".join((x.find("span").text).split()))
        #print("#####################################")
        if " ".join((x.find("span").text).split()) == word:
            return x
    return -1  

In [ ]:
nFolders = 299
c = 12022
url_file = urls
num_lines = sum(1 for line in open(url_file))
pagesPerFolder = 64
outputTSV = "dataset.tsv"

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
with open(outputTSV, 'a') as out_file:
    tsv_writer = csv.writer(out_file, delimiter= '\t')
    tsv_writer.writerow(["animeTitle", "animeType", "animeNumEpisode", "releaseDate", "endDate", "animeNumMembers", "animeScore", "animeUsers", "animeRank", "animePopularity", "animeDescription", "animeRelated", "animeCharacters", "animeVoices", "animeStaff"])
    for x in range(1, 20000 +1):

        number = int(c/(pagesPerFolder)) + 1
        path = path_1 + "/cartella{}".format(number) + "/page_{}.html".format(c+1)
        print("path: ", path)
        tsv_writer.writerow(extractData(path))
        c = c + 1

Streaming output truncated to the last 5000 lines.
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27023.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27024.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27025.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27026.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27027.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27028.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27029.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27030.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27031.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27032.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27033.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/cartella423/page_27034.html
path:  /content/drive/MyDrive/ADM-HW3/cartelle/ca

In [ ]:
outputTSV


'dataset.tsv'